In [1]:
H_dict = {'IIII': (-0.32760818967480926+0j), 'IIIZ': (-0.130362920571091+0j), 'IIZI': (-0.130362920571091+0j), 'IIZZ': (0.16326768673564335+0j), 'IZII': (0.13716572937099508+0j), 'IZIZ': (0.10622904490856078+0j), 'IZZI': (0.15542669077992832+0j), 'ZIII': (0.13716572937099508+0j), 'ZIIZ': (0.15542669077992832+0j), 'ZIZI': (0.10622904490856078+0j), 'ZZII': (0.15660062488237952+0j), 'XXYY': (-0.04919764587136754+0j), 'XYYX': (0.04919764587136754+0j), 'YXXY': (0.04919764587136754+0j), 'YYXX': (-0.04919764587136754+0j)}

In [2]:
import os
import json
import numpy as np
from symmer import PauliwordOp, QuantumState, QubitTapering
from symmer.operators import AntiCommutingOp, IndependentOp, PauliwordOp
from symmer.utils import exact_gs_energy
from symmer.evolution.exponentiation import *
from symmer.evolution.decomposition import *
from tomography.cst import ClassicalShadow
from qiskit import QuantumCircuit, execute, Aer
from qiskit.opflow import CircuitStateFn
from scipy.sparse import csr_array
from qiskit.circuit.library import PauliEvolutionGate

/opt/homebrew/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [3]:
Ham = PauliwordOp.from_dictionary(H_dict)
clique_cover = Ham.clique_cover()
print(clique_cover)

{0: -0.328+0.000j IIII +
-0.130+0.000j IIIZ +
-0.130+0.000j IIZI +
 0.163+0.000j IIZZ +
 0.137+0.000j IZII +
 0.106+0.000j IZIZ +
 0.155+0.000j IZZI +
 0.137+0.000j ZIII +
 0.155+0.000j ZIIZ +
 0.106+0.000j ZIZI +
 0.157+0.000j ZZII, 1: -0.049+0.000j XXYY +
 0.049+0.000j XYYX +
 0.049+0.000j YXXY +
-0.049+0.000j YYXX}


In [4]:
gs_nrg_tap, gs_psi_tap = exact_gs_energy(Ham.to_sparse_matrix)
print(gs_nrg_tap, gs_psi_tap)


-1.101150330232619  0.175+0.000j |0011> +
-0.985+0.000j |1100>


In [5]:
sp_circ = QuantumCircuit(gs_psi_tap.n_qubits)
gs_psi_tap_dict = gs_psi_tap.to_dictionary
gs_psi_tap_array = np.array([0]*(2**gs_psi_tap.n_qubits), dtype=complex)
def bin_to_int(b):
    l = len(b)
    val = 0
    for a in range(l):
        val += int(b[a]) * (2**(l-1-a))
    return val
for p, w in zip(list(gs_psi_tap_dict.keys()), list(gs_psi_tap_dict.values())):
    val = bin_to_int(p)
    gs_psi_tap_array[val] = w 
sp_circ.prepare_state(gs_psi_tap_array)


assert np.all(np.isclose(
    CircuitStateFn(sp_circ).to_matrix(),
    gs_psi_tap.to_sparse_matrix.toarray().reshape(-1)
))

In [6]:
from functools import reduce

shots = 1000

stabilizers_generators_bitstrings = []

for _ in range(shots):
    clique = np.random.choice(list(clique_cover.values()))
    independent_op = IndependentOp.clique_generators(clique)

    assert np.all(independent_op.adjacency_matrix)
    _, successful_mask = clique.generator_reconstruction(independent_op)
    assert np.all(successful_mask)

    # independent_op = IndependentOp.symmetry_generators(clique)
    independent_op.generate_stabilizer_rotations()
    stabilizer_rotations = independent_op.stabilizer_rotations

    rotation_operator = reduce(
        lambda x,y:x*y,
        [(P**0 + P*1j)*(1/np.sqrt(2)) for P,_ in stabilizer_rotations[::-1]],
        PauliwordOp.from_list(['I'*Ham.n_qubits])
    )
    
    qc = QuantumCircuit(gs_psi_tap.n_qubits)
    qc += sp_circ
    for rot, _ in stabilizer_rotations:
        p_rot = rot.to_qiskit
        evo = PauliEvolutionGate(p_rot, time=-np.pi/4)
        qc.append(evo, list(range(gs_psi_tap.n_qubits)))

    output = CircuitStateFn(qc).to_matrix()

    sample_probs = [np.abs(i)**2 for i in output.flatten()]
    shot = np.random.choice(range(2**gs_psi_tap.n_qubits), p=sample_probs)
    shot_bin = bin(shot)[2:].zfill(gs_psi_tap.n_qubits)
    
    assert np.all(np.isclose(
        output,
        (rotation_operator * gs_psi_tap).to_sparse_matrix.toarray().reshape(-1)
    ))

    stabilizers_generators_bitstrings.append((stabilizer_rotations, independent_op, shot_bin))

In [7]:
def parity(bitstring):
    """
    Returns 1 if the bitstring has an even number of 1s and returns 0 otherwise
    """
    parity = 1
    for bit in bitstring:
        if bit == "1":
            parity = parity * (-1)
    return parity

In [1]:
gs_nrg_cst = 0
terms = [PauliwordOp.from_dictionary({list(Ham.to_dictionary.keys())[i] : list(Ham.to_dictionary.values())[i]}) for i in range(len(list(Ham.to_dictionary.keys())))]

for term in terms:
    pauli_string = list(term.to_dictionary.keys())[0]
    weight = list(term.to_dictionary.values())[0]
    shots = 0
    counts0 = 0
    counts1 = 0
    for stabilizer_rotations, generators, bitstring in stabilizers_generators_bitstrings:
        rotated_term_dict = term.perform_rotations(stabilizer_rotations).to_dictionary
        rotated_term = list(rotated_term_dict.keys())[0]
        rotated_weight = list(rotated_term_dict.values())[0]

        flip_sign = False if rotated_weight == weight else True

        recon_matrix, successful_mask = term.generator_reconstruction(generators)

        Z_idxs = []
        for p_idx in range(len(rotated_term)):
            p = rotated_term[p_idx]
            if p == "Z":
                Z_idxs.append(p_idx)

        if successful_mask[0] and not flip_sign:
            shots += 1
            subbitstring = ""
            for idx in Z_idxs:
                subbitstring += bitstring[idx]
            if parity(subbitstring) == 1:
                counts1 += 1
            else:
                counts0 += 1
        elif successful_mask[0] and flip_sign:
            shots += 1
            subbitstring = ""
            for idx in Z_idxs:
                subbitstring += bitstring[idx]
            if parity(subbitstring) == 1:
                counts0 += 1
            else:
                counts1 += 1

    if shots > 0:
        gs_nrg_cst += (counts1/shots - counts0/shots) * weight
    
print("gs_nrg_tap_cst = ", str(gs_nrg_cst))


NameError: name 'Ham' is not defined